# Cluster errors to identify the type of errors that can appear in solver reports 

# Table of Contents

1. [Introduction](#Introduction)
2. [Import Packages](#Import_packages)
3. [Load the clean solver data saved by 'PreprocessSolverErrorData' notebook](#load_clean_data)
4. [Filter data using Solver / datetime](#filter)
5. [Word to Vector Conversion using Continuous Bag of Words model (CBOW)](#word2vec)
6. [Sentence (error message) to vector conversion](#sent2vec)
7. [Clustering using DBScan](#clustering)
8. [Get cluster statistics such as : "pattern", "mean_length", "mean_similarity"](#cluster_stats)
9. [Save clustered data to Ceph](#save_to_ceph)
10. [View data from each cluster](#view_data)
 1. [Cluster No. 0: FileNotFoundError](#c0)
 2. [Cluster No. 1: UnableToExecuteGccError](#c1)
 3. [Cluster No. 3: NoMatchingDistributionFoundError](#c3)
11. [Clusters with more than one error](#clusters_with_more_than_one_error)
 1. [Cluster No. 10: ImportError, HTTPError](#c10)
 2. [Cluster No. 106: CalledProcessError, FileNotFoundError, KeyError, RuntimeError](#c106)
 3. [Cluster No. 116:  ConnectionError, OSError, MaxRetryError, DistutilsError, ResponseError](#c16)
 4. [Cluster No. 7: CheckTheLogsError : Need further exploring](#c7) 

## Introduction  <a id='Introduction'></a>

The purpose of this notebook is to cluster solver errors so that we can derive context on why dependencies cannot be solved in order to better advise users on why something cannot be used.

#### Summary :
- Preprocessed data by [PreprocessSolverErrorData](/PreprocessSolverErrorData.ipynb) notebook is loaded.
- Each word in converted into a vector using [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) (Continuous Bag of Words model). 
- Each error message is then converted into a vector(Sentence2vec using word2vec model).
- Clustering is done using [DBScan](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).
- Cluster statistics such as "pattern", "mean_length" and "mean_similarity" is calculated.
- Error Class is defined and added to the dataframe.
- Saved the classified error data to Ceph.

## Import packages <a id='Import_packages'></a>

In [36]:
import pandas as pd
import multiprocessing
import pickle
import numpy as np
import difflib
import regex as re
import boto3
import os

from math import sqrt
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from gensim.models import Word2Vec
from kneed import KneeLocator
from string import punctuation    

In [2]:
pd.set_option('max_colwidth', 2600)
pd.set_option('display.max_rows', 200)

In [3]:
cpu_number = multiprocessing.cpu_count()
w2v_window= 7

## Load the clean solver data saved by 'PreprocessSolverErrorData' notebook <a id='load_clean_data'></a>

In [4]:
entire_error_df = pd.read_csv('error-clean-data.csv')

In [5]:
len(entire_error_df)

93532

## Filter data using Solver / datetime <a id='filter'></a>

In [6]:
def filter_data(entire_error_df, solver_name=None, start_date='2019-12-27',end_date='2020-01-14', mode='solver'):
    if mode == 'solver':
        error_df = entire_error_df.loc[entire_error_df['solver'] == solver_name]
    elif mode == 'datetime':
        mask = (entire_error_df['datetime'] >= start_date) & (entire_error_df['datetime'] <= end_date)
        error_df = entire_error_df.loc[mask]
    elif mode == 'all':
        error_df = entire_error_df
    return error_df

In [7]:
entire_error_df['solver'].unique()

array(['solver-fedora-31-py37', 'solver-fedora-31-py38',
       'solver-fedora-32-py37', 'solver-fedora-32-py38',
       'solver-rhel-8-py36'], dtype=object)

In [8]:
#error_df = filter_data(entire_error_df, solver_name = 'solver-fedora-31-py37', mode='solver')
#error_df = filter_data(entire_error_df, start_date='2019-12-24',end_date='2020-01-14', mode='datetime')
error_df = filter_data(entire_error_df, mode = 'all')

In [9]:
len(error_df)

93532

### Extract tokenized_clustering_data for clustering

In [10]:
clean_clustering_data = error_df['tokenized_clustering_data']

## Word to Vector Conversion using Continuous Bag of Words model (CBOW) <a id='word2vec'></a>

In [11]:
print('Number of rows in training data :', len(clean_clustering_data))

Number of rows in training data : 93532


In [12]:
def detect_embedding_size(tokens):
    flat_list = [item for row in tokens for item in row]
    vocab = set(flat_list)
    embedding_size = round(len(vocab) ** (2/3))
    if embedding_size >= 400:
        embedding_size = 400
    return embedding_size

w2v_size = detect_embedding_size(clean_clustering_data)

In [13]:
def tokens_vectorization(clustering_data, w2v_size, w2v_window, cpu_number, model_name):
    iterations = 100
    word2vec = Word2Vec(clustering_data,
                           size = w2v_size, 
                           window = w2v_window, 
                           min_count=1, 
                           workers = cpu_number,
                           iter=iterations)
    word2vec.save(model_name)
    return word2vec

In [42]:
word2vec = tokens_vectorization(clean_clustering_data, 
                                 w2v_size = w2v_size, 
                                 w2v_window= w2v_window, 
                                 cpu_number = cpu_number, 
                                 model_name='../models/word2vec.model')

2020-08-11 00:53:18,672 [27719] WARNING  gensim.models.base_any2vec:720: [JupyterRequire] consider setting layer size to a multiple of 4 for greater performance
2020-08-11 00:53:18,673 [27719] WARNING  gensim.models.word2vec:1378: [JupyterRequire] Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.


## Sentence (error message) to vector conversion <a id='sent2vec'></a>

sum all content words in the documents and divide by the number of content words.

In [15]:
def sentence_vectorization(clustering_data, word2vec):
    sent2vec = []
    for sent in clustering_data:
        sent_vec = []
        numw = 0
        for w in sent:
            try:
                sent_vec = word2vec[w] if numw == 0 else np.add(sent_vec, word2vec[w])
                numw += 1
            except Exception:
                pass
        sent2vec.append(np.asarray(sent_vec) / numw)   
    return np.vstack(sent2vec)

In [16]:
sent2vec = sentence_vectorization(clean_clustering_data, word2vec)

2020-08-11 00:38:05,448 [27719] WARNING  py.warnings:99: [JupyterRequire] /home/skotak/.local/share/virtualenvs/solver-error-classifier-SslAmjTI/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  



## Clustering using DBScan  <a id='clustering'></a>

Based on a set of points DBSCAN groups together points that are close to each other based on a distance measurement(epsilon) and a minimum number of points. It also marks as outliers the points that are in low-density regions.

Find the avg_distances using NearestNeighbors between the data points.

In [17]:
def kneighbors(sent2vec):
    k = round(sqrt(len(sent2vec)))
    neigh = NearestNeighbors(n_neighbors=k)
    nbrs = neigh.fit(sent2vec)
    distances, indices = nbrs.kneighbors(sent2vec)
    distances = [np.mean(d) for d in np.sort(distances, axis=0)]
    return distances

avg_distances = kneighbors(sent2vec)

Calculate epsilon, which is the linkage distance threshold above which, clusters will not be merged.

In [18]:
def epsilon_search(distances):
    kneedle = KneeLocator(distances, list(range(len(distances))))
    epsilon = max(kneedle.all_elbows) if (len(kneedle.all_elbows) > 0) else 1
    return epsilon

In [19]:
epsilon = epsilon_search(avg_distances)

DBScan Clustering using epsilon and min_samples as 1

In [20]:
def dbscan(epsilon, min_samples, cpu_number, sent2vec):
    cluster_labels = DBSCAN(eps=epsilon,
                            min_samples= min_samples,
                            n_jobs=cpu_number).fit_predict(sent2vec)
    return cluster_labels

In [21]:
#cluster_labels = hierarchical(epsilon, sent2vec)
cluster_labels = dbscan(epsilon, 1, cpu_number, sent2vec)

In [22]:
len(cluster_labels)

93532

In [23]:
error_df['cluster_no.'] = cluster_labels

## Get cluster statistics such as : "pattern", "mean_length", "mean_similarity" <a id='cluster_stats'></a>

In [24]:
def clustered_output(error_df, mode='INDEX'):
    groups, unique_rows = {}, {}
    for key, value in error_df.groupby(['cluster_no.']):
        unique_rows[str(key)] = set(value['clustering_data'])
        if mode == 'ALL':
            groups[str(key)] = value.to_dict(orient='records')
        elif mode == 'Tokenized':
            groups[str(key)] = value['tokenized_clustering_data'].values.tolist()
        elif mode == 'CLEANED':
            groups[str(key)] = value['clustering_data'].values.tolist()
    return groups, unique_rows

In [25]:
table = str.maketrans(punctuation, ' '*len(punctuation))

def find_matching_blocks(strings):
    curr = strings[0]
    curr = curr.replace('ERROR', '')
    curr = curr.replace('Command exited with non-zero status code (1):', '')
    if len(strings) == 1:
        #return curr
        return curr.translate(table).strip()
    else:
        cnt = 1
        for i in range(cnt, len(strings)):
            matches = difflib.SequenceMatcher(None, curr, strings[i])
            common = []
            for match in matches.get_matching_blocks():
                common.append(curr[match.a:match.a + match.size])
            curr = ''.join(common)
            cnt = cnt + 1
            if cnt == len(strings):
                break
        if curr == '':
            'NO COMMON PATTERNS HAVE BEEN FOUND'
        #return curr
        return curr.translate(table).strip()

def get_similarity(rows):
    s = []
    for i in range(0, len(rows)):
        s.append(difflib.SequenceMatcher(None, rows[0], rows[i]).ratio() * 100)
    return s

In [26]:
STATISTICS = ["cluster_name", "cluster_size", "pattern", 'CLASS', "mean_similarity"]

def statistics(error_df, output_mode='frame'):
    """
    Returns dictionary with statistic for all clusters
    "cluster_name" - name of a cluster
    "cluster_size" = number of log messages in cluster
    "pattern" - all common substrings in messages in the cluster
    "mean_length" - average length of log messages in cluster
    "mean_similarity" - average similarity of log messages in cluster
    (calculated as the levenshtein distances between the 1st and all other log messages)
    :param clustered_df:
    :param output_mode: frame | dict
    :return:
    """
    clusters = []
    clustered_df, unique_rows = clustered_output(error_df, mode='CLEANED')
    clustered_df_class, unique_rows = clustered_output(error_df, mode='Tokenized')
    for item in clustered_df:
        row = clustered_df[item]
        matcher = find_matching_blocks(row)
        class_matcher = find_matching_blocks(clustered_df_class[item])
        similarity = get_similarity(row)
        clusters.append([item,
                         len(row),
                         matcher,
                         class_matcher,
                         #unique_rows[item],
                         #np.mean(lengths),
                         np.mean(similarity)])
    df = pd.DataFrame(clusters, columns=STATISTICS).round(2).sort_values(by='cluster_size', ascending=False)
    if output_mode == 'frame':
        return df
    else:
        return df.to_dict(orient='records')

In [27]:
stat = statistics(error_df, output_mode='frame')
stat_df = pd.DataFrame.from_dict(stat)

In [28]:
print('Number of clusters : ', len(stat_df))

Number of clusters :  125


Generate CLASS label

In [29]:
def get_class_label(stat_df):
    class_labels = []
    number_of_errors = []
    MachineDefinedError = []
    for item in stat_df['CLASS']:
        if "Error" in item.split():
            item = item.replace('Error', '')
        row = item.split()
        #if len(row) > 1 and len(re.findall(r'Error', str(row))) < 2:
        if not re.search('(\w\w*Error)', item):
            MachineDefinedError.append('NO')
            item = ''
            for word in row:
                item += word[0].upper() + word[1:]
            item += "Error"
        else:
            if len(re.findall(r'Error', str(row))) > 1:
                item = ', '.join(row)
            else:
                item = ''.join(row)
            MachineDefinedError.append('YES')
        class_labels.append(item)
        number_of_errors.append(len(re.findall(r'Error', str(item))))
    return class_labels, number_of_errors, MachineDefinedError

In [30]:
class_labels, number_of_errors, MachineDefinedError = get_class_label(stat_df)

In [31]:
stat_df['number_of_errors'] = number_of_errors
stat_df['MachineDefinedError?'] = MachineDefinedError
stat_df['CLASS'] = class_labels

In [32]:
stat_df.sort_values(by='cluster_size', ascending=False)

,cluster_name,cluster_size,pattern,CLASS,mean_similarity,number_of_errors,MachineDefinedError?
3,3,20616,No matching distribution found for,NoMatchingDistributionFoundError,77.04,1,NO
4,4,15965,SyntaxError,SyntaxError,100.00,1,YES
1,1,13075,unable to execute gcc No such file or directory,UnableToExecuteGccError,100.00,1,NO
5,5,10686,ModuleNotFoundError,ModuleNotFoundError,100.00,1,YES
0,0,8539,Fileotoundo,FileNotFoundError,99.61,1,YES
2,2,4499,NameError,NameError,100.00,1,YES
8,8,4182,Failed building wheel for,FailedBuildingWheelError,85.70,1,NO
7,7,3658,rrore t,CheckTheLogsError,96.49,1,NO
11,11,1868,AttributeError,AttributeError,100.00,1,YES
10,10,1337,HTTPError ImportError,"HTTPError, ImportError",100.00,2,YES


In [33]:
error_df['CLASS'] = error_df['cluster_no.'].map(stat_df['CLASS'])
error_df['number_of_errors'] = error_df['cluster_no.'].map(stat_df['number_of_errors'])
error_df['MachineDefinedError?'] = error_df['cluster_no.'].map(stat_df['MachineDefinedError?'])

## Save clustered data to Ceph <a id='save_to_ceph'></a>

In [38]:
import os
os.environ['THOTH_S3_ENDPOINT_URL'] = 'https://s3.upshift.redhat.com/'
os.environ['AWS_ACCESS_KEY_ID'] = '***********************'
os.environ['AWS_SECRET_ACCESS_KEY'] = '***********************'

In [39]:
from io import StringIO

def store_csv_to_ceph(error_df):
    csv_buffer = StringIO()
    error_df = error_df.drop(columns =['index', 'message','split_message', 'Error_info', 'command_info', 
                                       'cwd', 'Complete_output','ERROR', 'Exception', 'specific_error'])
    error_df.to_csv(csv_buffer, header=False, sep ='`', index=False)
    bucket = 'DH-PLAYPEN'
    s3_resource = boto3.resource('s3',
                        endpoint_url= os.environ['THOTH_S3_ENDPOINT_URL'],
                        aws_access_key_id = os.environ["AWS_ACCESS_KEY_ID"],
                        aws_secret_access_key= os.environ['AWS_SECRET_ACCESS_KEY'])
    s3_resource.Object(bucket, 'thoth/data/solver-error-context/solver-error-context.csv').put(Body=csv_buffer.getvalue())

In [40]:
store_csv_to_ceph(error_df)

## View data from each cluster <a id='view_data'></a>

In [41]:
def get_data_from_cluster(df_processed, clusters, cluster_number):
    indices = [i for i, x in enumerate(clusters) if x == cluster_number]
    df_grouped = df_processed.iloc[indices]
    print(len(df_grouped))
    return df_grouped

def split_log(log_messages):
    log_messages = log_messages.split('\n')
    return log_messages

### Cluster No. 0: FileNotFoundError <a id='c0'></a>

In [40]:
get_data_from_cluster(error_df, cluster_labels, 0)[['package_name', 'package_version', 'solver','message', 
                                                    'specific_error', 'CLASS', 'MachineDefinedError?']]

8539


,package_name,package_version,solver,message,specific_error,CLASS,MachineDefinedError?
0,json2html,1.1.0,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info\n cwd: /tmp/pip-install-1eq417ji/json2html/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>\n long_description=open('README.md').read(),\n FileNotFoundError: [Errno 2] No such file or directory: 'README.md'\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n","[""FileNotFoundError: [Errno 2] No such file or directory: 'README.md'""]",FileNotFoundError,YES
8,pyobjc-framework-storekit,5.1.1,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-jhjhpfpk/pyobjc-framework-storekit/setup.py'""'""'; __file__='""'""'/tmp/pip-install-jhjhpfpk/pyobjc-framework-storekit/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-jhjhpfpk/pyobjc-framework-storekit/pip-egg-info\n cwd: /tmp/pip-install-jhjhpfpk/pyobjc-framework-storekit/\n Complete output (15 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-jhjhpfpk/pyobjc-framework-storekit/setup.py"", line 26, in <module>\n for fn in os.listdir('Modules')\n File ""/tmp/pip-install-jhjhpfpk/pyobjc-framework-storekit/pyobjc_setup.py"", line 389, in Extension\n os_level = get_os_level()\n File ""/tmp/pip-install-jhjhpfpk/pyobjc-framework-storekit/pyobjc_setup.py"", line 203, in get_os_level\n pl = plistlib.readPlist('/System/Library/CoreServices/SystemVersion.plist')\n File ""/usr/lib64/python3.7/plistlib.py"", line 99, in readPlist\n with _maybe_open(pathOrFile, 'rb') as fp:\n File ""/usr/lib64/python3.7/contextlib.py"", line 112, in __enter__\n return next(self.gen)\n File ""/usr/lib64/python3.7/plistlib.py"", line 82, in _maybe_open\n with open(pathOrFile, mode) as fp:\n FileNotFoundError: [Errno 2] No such file or directory: '/System/Library/CoreServices/SystemVersion.plist'\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n","[""FileNotFoundError: [Errno 2] No such file or directory: '/System/Library/CoreServices/SystemVersion.plist'""]",FileNotFoundError,YES
9,symsynd,0.4.0,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-d9sdtlr3/symsynd/setup.py'""'""'; __file__='""'""'/tmp/pip-install-d9sdtlr3/symsynd/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-d9sdtlr3/symsynd/pip-egg-info\n cwd: /tmp/pip-install-d9sdtlr3/symsynd/\n Complete output (31 lines):\n WARNING: The directory '/.cache/pip/http' or its par

### Cluster No. 1: UnableToExecuteGccError	<a id='c1'></a>

In [28]:
get_data_from_cluster(error_df, cluster_labels, 1)[['package_name', 'package_version', 'solver','message', 
                                                    'specific_error', 'CLASS', 'MachineDefinedError?']]

13075


,package_name,package_version,solver,message,specific_error,CLASS,MachineDefinedError?
1,guillotina,1.3.0,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""'; __file__='""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-l0mjgi64 --python-tag cp37\n cwd: /tmp/pip-install-5y8hsn2x/guillotina/\n Complete output (308 lines):\n running bdist_wheel\n running build\n running build_py\n creating build\n creating build/lib.linux-x86_64-3.7\n creating build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/__init__.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/addons.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/annotations.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/async.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/blob.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/browser.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/constraintypes.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/content.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/contentnegotiation.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/cors.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/directives.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/event.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/events.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/exceptions.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/files.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/glogging.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/i18n.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/jose.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/languages.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/patch.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/permissions.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/registry.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/renderers.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guillotina/subscribers.py -> build/lib.linux-x86_64-3.7/guillotina\n copying guill...","[""unable to execute 'gcc': No such file or directory""]",UnableToExecuteGccError,NO
2,documenttemplate,2.13.4,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-rfhgqa6g/documenttemplate/setup.py'""'""'; __file__='""'""'/tmp/pip-install-rfhgqa6g/documenttemplate/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-0oa5n_91 --python-tag cp37\n cwd: /tmp/pip-install-rfhgqa6g/documenttemplate/\n Complete output (76 lines):\n running bdist_wheel\n running build\n running build_py\n creating build\n creating build/lib.linux-x86_64-3.7\n creating build/lib.linux-x86_64-3.7/DocumentTemplate\n copying src/DocumentTemplate/__init__.py -> build/lib.linux-x86_64-3.7/DocumentTemplate\n copying src/DocumentTemplate/_DocumentTemplate.py -> build/lib.linux-x86_64-3.7/DocumentTemplate\n copying src/DocumentTemplate/DT_HTML.py -> build/lib.linux-x86_64-3.7/DocumentTemplate\n copying src/DocumentTemplate/DT_If.py -> build/lib.linux-x86

### Cluster No. 3: NoMatchingDistributionFoundError <a id='c3'></a>

In [29]:
get_data_from_cluster(error_df, cluster_labels, 3)[['package_name', 'package_version', 'solver','message', 
                                                    'ERROR', 'CLASS', 'MachineDefinedError?']]

20616


,package_name,package_version,solver,message,ERROR,CLASS,MachineDefinedError?
4,mxnet-mkl,1.7.0b20200309,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement mxnet-mkl==1.7.0b20200309 (from versions: 0.10.0, 0.10.0.post2, 0.11.0b20170820, 0.11.0, 0.11.1b20170828, 0.11.1b20170906, 0.11.1b20170913, 0.11.1b20170920, 0.11.1b20170927, 0.11.1b20171004, 0.11.1b20171011, 0.12.0b20171018, 0.12.0, 0.12.1b20171105, 0.12.1b20171119, 0.12.1b20171126, 0.12.1b20171203, 0.12.1, 1.0.0b20171210, 1.0.0, 1.0.0.post0, 1.0.0.post1, 1.0.0.post2, 1.0.0.post4, 1.0.1b20171231, 1.0.1b20180107, 1.0.1b20180114, 1.0.1b20180121, 1.0.1b20180128, 1.1.0b20180208, 1.1.0b20180215, 1.1.0, 1.2.0b20180320, 1.2.0b20180327, 1.2.0b20180403, 1.2.0b20180410, 1.2.0b20180417, 1.2.0b20180424, 1.2.0b20180501, 1.2.0b20180508, 1.2.0b20180515, 1.2.0b20180522, 1.2.0, 1.2.1, 1.2.1.post0, 1.2.1.post1, 1.3.0b20180529, 1.3.0b20180606, 1.3.0b20180613, 1.3.0b20180620, 1.3.0b20180627, 1.3.0b20180704, 1.3.0b20180711, 1.3.0b20180718, 1.3.0b20180725, 1.3.0b20180805, 1.3.0b20180810, 1.3.0b20180819, 1.3.0b20180820, 1.3.0b20180831, 1.3.0b20180909, 1.3.0b20180915, 1.3.0, 1.3.0.post0, 1.3.1b20180920, 1.3.1b20180927, 1.3.1b20181004, 1.3.1b20181014, 1.3.1b20181021, 1.3.1b20181027, 1.3.1b20181104, 1.3.1b20181111, 1.3.1b20181118, 1.3.1, 1.4.0b20181125, 1.4.0b20181202, 1.4.0, 1.4.0.post0, 1.4.1, 1.5.0b20181207, 1.5.0b20181216, 1.5.0b20181223, 1.5.0b20181227, 1.5.0b20190106, 1.5.0b20190112, 1.5.0b20190120, 1.5.0b20190127, 1.5.0b20190203, 1.5.0b20190210, 1.5.0b20190217, 1.5.0b20190224, 1.5.0b20190303, 1.5.0b20190310, 1.5.0b20190314, 1.5.0b20190331, 1.5.0b20190401, 1.5.0b20190409, 1.5.0b20190428, 1.5.0b20190505, 1.5.0b20190512, 1.5.0b20190519, 1.5.0b20190526, 1.5.0b20190602, 1.5.0b20190609, 1.5.0b20190616, 1.5.0b20190617, 1.5.0b20190618, 1.5.0b20190619, 1.5.0b20190620, 1.5.0b20190621, 1.5.0b20190622, 1.5.0b20190623, 1.5.0b20190624, 1.5.0b20190625, 1.5.0b20190626, 1.5.0b20190627, 1.5.0b20190628, 1.5.0b20190629, 1.5.0b20190630, 1.5.0b20190701, 1.5.0b20190702, 1.5.0b20190703, 1.5.0b20190704, 1.5.0b20190705, 1.5.0b20190706, 1.5.0b20190707, 1.5.0b20190708, 1.5.0b20190709, 1.5.0b20190710, 1.5.0b20190711, 1.5.0b20190712, 1.5.0b20190713, 1.5.0b20190714, 1.5.0b20190715, 1.5.0b20190716, 1.5.0b20190717, 1.5.0b20190718, 1.5.0b20190719, 1.5.0b20190720, 1.5.0b20190721, 1.5.0b20190722, 1.5.0b20190723, 1.5.0, 1.5.1, 1.5.1.post0, 1.6.0b20190724, 1.6.0b20190725, 1.6.0b20190726, 1.6.0b20190727, 1.6.0b20190728, 1.6.0b20190729, 1.6.0b20190730, 1.6.0b20190731, 1.6.0b20190801, 1.6.0b20190807, 1.6.0b20190808, 1.6.0b20190809, 1.6.0b20190810, 1.6.0b20190812, 1.6.0b20190813, 1.6.0b20190814, 1.6.0b2...",['ERROR: No matching distribution found for mxnet-mkl==1.7.0b20200309'],NoMatchingDistributionFoundError,NO
12,cbapi,0.9.0,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement cbapi==0.9.0 (from versions: 0.8.0, 0.8.1, 0.9.1, 0.9.2, 0.9.3, 0.9.4, 0.9.5, 0.9.6, 0.9.7, 0.9.8, 1.0.0, 1.0.1, 1.1.0, 1.1.1, 1.2.0, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.3.4, 1.3.5, 1.3.6, 1.4.0, 1.4.1, 1.4.2, 1.4.3, 1.4.4, 1.4.5, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 1.5.4, 1.5.5, 1.5.6, 1.6.0)\nERROR: No matching distribution found for cbapi==0.9.0\n",['ERROR: No matching distribution found for cbapi==0.9.0'],NoMatchingDistributionFoundError,NO
19,lupa,0.2,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement lupa==0.2 (from versions: 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.10, 0.11, 0.12, 0.13, 0.13.1, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 1.0b1, 1.0, 1.0.1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9)\nERROR: No matching distribution found for lupa==0.2\n",['ERROR: No matching distribution found for lupa==0.2'],NoMatchingDistributionFoundError,NO
36,mxnet,1.6.0b20200224,solver-fedora-31-py37,"Command exited with non-zero status code (

## Clusters with more than one error  <a id='clusters_with_more_than_one_error'></a>

### Cluster No. 10: ImportError, HTTPError <a id='c10'></a>

In [30]:
get_data_from_cluster(error_df, cluster_labels, 10)[['package_name', 'package_version', 'solver','message', 
                                                    'specific_error', 'CLASS', 'MachineDefinedError?']]

1337


,package_name,package_version,solver,message,specific_error,CLASS,MachineDefinedError?
33,coverage,3.5.1b1,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-c7qbxce0/coverage/setup.py'""'""'; __file__='""'""'/tmp/pip-install-c7qbxce0/coverage/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-c7qbxce0/coverage/pip-egg-info\n cwd: /tmp/pip-install-c7qbxce0/coverage/\n Complete output (31 lines):\n Downloading http://pypi.python.org/packages/source/d/distribute/distribute-0.6.13.tar.gz\n Traceback (most recent call last):\n File ""/tmp/pip-install-c7qbxce0/coverage/distribute_setup.py"", line 143, in use_setuptools\n raise ImportError\n ImportError\n \n During handling of the above exception, another exception occurred:\n \n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-c7qbxce0/coverage/setup.py"", line 49, in <module>\n use_setuptools()\n File ""/tmp/pip-install-c7qbxce0/coverage/distribute_setup.py"", line 145, in use_setuptools\n return _do_download(version, download_base, to_dir, download_delay)\n File ""/tmp/pip-install-c7qbxce0/coverage/distribute_setup.py"", line 124, in _do_download\n to_dir, download_delay)\n File ""/tmp/pip-install-c7qbxce0/coverage/distribute_setup.py"", line 193, in download_setuptools\n src = urlopen(url)\n File ""/usr/lib64/python3.7/urllib/request.py"", line 222, in urlopen\n return opener.open(url, data, timeout)\n File ""/usr/lib64/python3.7/urllib/request.py"", line 531, in open\n response = meth(req, response)\n File ""/usr/lib64/python3.7/urllib/request.py"", line 641, in http_response\n 'http', request, response, code, msg, hdrs)\n File ""/usr/lib64/python3.7/urllib/request.py"", line 569, in error\n return self._call_chain(*args)\n File ""/usr/lib64/python3.7/urllib/request.py"", line 503, in _call_chain\n result = func(*args)\n File ""/usr/lib64/python3.7/urllib/request.py"", line 649, in http_error_default\n raise HTTPError(req.full_url, code, msg, hdrs, fp)\n urllib.error.HTTPError: HTTP Error 403: SSL is required\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n","['raise ImportError', 'ImportError', 'raise HTTPError(req.full_url, code, msg, hdrs, fp)', 'urllib.error.HTTPError: HTTP Error 403: SSL is required']","ImportError, HTTPError",YES
154,pyhull,1.3.8,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-b49kv_qn/pyhull/setup.py'""'""'; __file__='""'""'/tmp/pip-install-b49kv_qn/pyhull/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-b49kv_qn/pyhull/pip-egg-info\n cwd: /tmp/pip-install-b49kv_qn/pyhull/\n Complete output (31 lines):\n Downloading http://pypi.python.org/packages/source/d/distribute/distribute-0.6.10.tar.gz\n Traceback (most recent call last):\n File ""/tmp/pip-install-b49kv_qn/pyhull/distribute_setup.py"", line 143, in use_setuptools\n raise ImportError\n ImportError\n \n During handling of the above exception, another exception occurred:\n \n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-b49kv_qn/pyhull/setup.py"", line 7, in <module>\n use_setuptools(version='0.6.10')\n File ""/tmp/pip-install-b49kv_qn/pyhull/distribute_setup.py"", line 145

### An example of log from cluster 10

In [31]:
split_log(error_df['message'][33])

['Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:',
 '     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-c7qbxce0/coverage/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-c7qbxce0/coverage/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-install-c7qbxce0/coverage/pip-egg-info',
 '         cwd: /tmp/pip-install-c7qbxce0/coverage/',
 '    Complete output (31 lines):',
 '    Downloading http://pypi.python.org/packages/source/d/distribute/distribute-0.6.13.tar.gz',
 '    Traceback (most recent call last):',
 '      File "/tmp/pip-install-c7qbxce0/coverage/distribute_setup.py", line 143, in use_setuptools',
 '        raise ImportError',
 '    ImportError',
 '    ',
 '    During handling

### Cluster No. 106: CalledProcessError, FileNotFoundError, KeyError, RuntimeError <a id='c106'></a>

In [39]:
get_data_from_cluster(error_df, cluster_labels, 106)[['package_name', 'package_version', 'solver','message', 
                                                    'specific_error', 'CLASS', 'MachineDefinedError?']]

20


,package_name,package_version,solver,message,specific_error,CLASS,MachineDefinedError?
31434,pyicu,2.4.2,solver-fedora-31-py38,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-9ajst693/pyicu/setup.py'""'""'; __file__='""'""'/tmp/pip-install-9ajst693/pyicu/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-9ajst693/pyicu/pip-egg-info\n cwd: /tmp/pip-install-9ajst693/pyicu/\n Complete output (53 lines):\n Package icu-i18n was not found in the pkg-config search path.\n Perhaps you should add the directory containing `icu-i18n.pc'\n to the PKG_CONFIG_PATH environment variable\n Package 'icu-i18n', required by 'virtual:world', not found\n Traceback (most recent call last):\n File ""/tmp/pip-install-9ajst693/pyicu/setup.py"", line 62, in <module>\n ICU_VERSION = os.environ['ICU_VERSION']\n File ""/home/solver/venv/lib64/python3.8/os.py"", line 673, in __getitem__\n raise KeyError(key) from None\n KeyError: 'ICU_VERSION'\n \n During handling of the above exception, another exception occurred:\n \n Traceback (most recent call last):\n File ""/tmp/pip-install-9ajst693/pyicu/setup.py"", line 65, in <module>\n ICU_VERSION = check_output(('icu-config', '--version')).strip()\n File ""/tmp/pip-install-9ajst693/pyicu/setup.py"", line 18, in check_output\n return subprocess_check_output(popenargs)\n File ""/usr/lib64/python3.8/subprocess.py"", line 411, in check_output\n return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,\n File ""/usr/lib64/python3.8/subprocess.py"", line 489, in run\n with Popen(*popenargs, **kwargs) as process:\n File ""/usr/lib64/python3.8/subprocess.py"", line 854, in __init__\n self._execute_child(args, executable, preexec_fn, close_fds,\n File ""/usr/lib64/python3.8/subprocess.py"", line 1702, in _execute_child\n raise child_exception_type(errno_num, err_msg, err_filename)\n FileNotFoundError: [Errno 2] No such file or directory: 'icu-config'\n \n During handling of the above exception, another exception occurred:\n \n Traceback (most recent call last):\n File ""/tmp/pip-install-9ajst693/pyicu/setup.py"", line 68, in <module>\n ICU_VERSION = check_output(('pkg-config', '--modversion', 'icu-i18n')).strip()\n File ""/tmp/pip-install-9ajst693/pyicu/setup.py"", line 18, in check_output\n ...","['raise KeyError(key) from None', ""KeyError: 'ICU_VERSION'"", ""FileNotFoundError: [Errno 2] No such file or directory: 'icu-config'"", 'raise CalledProcessError(retcode, process.args,', ""subprocess.CalledProcessError: Command '('pkg-config', '--modversion', 'icu-i18n')' returned non-zero exit status 1."", ""raise RuntimeError('''"", 'RuntimeError: Please install pkg-config on your system or set the ICU_VERSION environment']","CalledProcessError, FileNotFoundError, KeyError, RuntimeError",YES
39250,pyicu,2.4,solver-fedora-31-py38,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-o1mz8wg3/pyicu/setup.py'""'""'; __file__='""'""'/tmp/pip-install-o1mz8wg3/pyicu/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-o1mz8wg3/pyicu/pip-egg-info\n cwd: /tmp/pip-install-o1mz8wg3/pyicu/\n Complete output (53 lines):\n Package icu-i18n was not found in the pkg-config search path.\n Perhaps you should add the directory containing `icu-i18n.pc'\n to the PKG_CONFIG_PATH environment variable\n Package 'icu-i18n', required by 'virtual:world', not found\n Traceback (most

### An example of log from cluster 106

In [33]:
split_log(error_df['message'][31434])

['Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:',
 '     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-9ajst693/pyicu/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-9ajst693/pyicu/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-install-9ajst693/pyicu/pip-egg-info',
 '         cwd: /tmp/pip-install-9ajst693/pyicu/',
 '    Complete output (53 lines):',
 '    Package icu-i18n was not found in the pkg-config search path.',
 "    Perhaps you should add the directory containing `icu-i18n.pc'",
 '    to the PKG_CONFIG_PATH environment variable',
 "    Package 'icu-i18n', required by 'virtual:world', not found",
 '    Traceback (most recent call last):',
 '      File "/tmp/pip-install-

### Cluster No. 116:  ConnectionError, OSError, MaxRetryError, DistutilsError, ResponseError <a id='c116'></a>

In [40]:
get_data_from_cluster(error_df, cluster_labels, 116)[['package_name', 'package_version', 'solver','message', 
                                                    'specific_error', 'CLASS', 'MachineDefinedError?']]

1


,package_name,package_version,solver,message,specific_error,CLASS,MachineDefinedError?
71802,girder-autojoin,3.0.7.dev11,solver-rhel-8-py36,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-03k22h60/girder-autojoin/setup.py'""'""'; __file__='""'""'/tmp/pip-install-03k22h60/girder-autojoin/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-03k22h60/girder-autojoin/pip-egg-info\n cwd: /tmp/pip-install-03k22h60/girder-autojoin/\n Complete output (110 lines):\n WARNING: Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba041048>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /packages/05/97/dd99fa9c0d9627a7b3c103a00f1566d8193aca8d473884ed258cca82b06f/setuptools_git-1.2-py2.py3-none-any.whl\n WARNING: Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba0411d0>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /packages/05/97/dd99fa9c0d9627a7b3c103a00f1566d8193aca8d473884ed258cca82b06f/setuptools_git-1.2-py2.py3-none-any.whl\n WARNING: Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba041278>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /packages/05/97/dd99fa9c0d9627a7b3c103a00f1566d8193aca8d473884ed258cca82b06f/setuptools_git-1.2-py2.py3-none-any.whl\n WARNING: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba041748>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /packages/05/97/dd99fa9c0d9627a7b3c103a00f1566d8193aca8d473884ed258cca82b06f/setuptools_git-1.2-py2.py3-none-any.whl\n WARNING: Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba041828>: Failed to establish a new connec...","['OSError: [Errno 101] Network is unreachable', 'pip._vendor.urllib3.exceptions.NewConnectionError: <pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba041a20>: Failed to establish a new connection: [Errno 101] Network is unreachable', 'raise MaxRetryError(_pool, url, error or ResponseError(cause))', ""pip._vendor.urllib3.exceptions.MaxRetryError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/05/97/dd99fa9c0d9627a7b3c103a00f1566d8193aca8d473884ed258cca82b06f/setuptools_git-1.2-py2.py3-none-any.whl (Caused by NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba041a20>: Failed to establish a new connection: [Errno 101] Network is unreachable',))"", 'raise ConnectionError(e, request=request)', ""pip._vendor.requests.exceptions.ConnectionError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/05/97/dd99fa9c0d9627a7b3c103a00f1566d8193aca8d473884ed258cca82b06f/setuptools_git-1.2-py2.py3-none-any.whl (Caused by NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba041a20>: Failed to establish a new connection: [Errno 101] Network is unreachable',))"", '

### An example of log from cluster 116

In [35]:
split_log(error_df['message'][71802])

['Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:',
 '     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-03k22h60/girder-autojoin/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-03k22h60/girder-autojoin/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-install-03k22h60/girder-autojoin/pip-egg-info',
 '         cwd: /tmp/pip-install-03k22h60/girder-autojoin/',
 '    Complete output (110 lines):',
 "      WARNING: Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f30ba041048>: Failed to establish a new connection: [Errno 101] Network is unre

### Cluster No. 7: CheckTheLogsError : Need further exploring <a id='c7'></a>

In [37]:
get_data_from_cluster(error_df, cluster_labels, 7)[['package_name', 'package_version', 'solver','message', 
                                                    'ERROR', 'CLASS', 'MachineDefinedError?']]

3658


,package_name,package_version,solver,message,ERROR,CLASS,MachineDefinedError?
18,astropy,1.1.post2,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-0_60gcb_/astropy/setup.py'""'""'; __file__='""'""'/tmp/pip-install-0_60gcb_/astropy/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-0_60gcb_/astropy/pip-egg-info\n cwd: /tmp/pip-install-0_60gcb_/astropy/\n Complete output (20 lines):\n running egg_info\n creating /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info\n writing /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info/PKG-INFO\n writing dependency_links to /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info/dependency_links.txt\n writing entry points to /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info/entry_points.txt\n writing requirements to /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info/requires.txt\n writing top-level names to /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info/top_level.txt\n writing manifest file '/tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info/SOURCES.txt'\n \n The C compiler used to compile Python gcc -pthread, and\n which is normally used to compile C extensions, is not\n available. You can explicitly specify which compiler to\n use by setting the CC environment variable, for example:\n \n CC=gcc python setup.py <command>\n \n or if you are using MacOS X, you can try:\n \n CC=clang python setup.py <command>\n \n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n",['ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.'],CheckTheLogsError,NO
122,txtorcon,0.8.2,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-v20lskkz/txtorcon/setup.py'""'""'; __file__='""'""'/tmp/pip-install-v20lskkz/txtorcon/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-v20lskkz/txtorcon/pip-egg-info\n cwd: /tmp/pip-install-v20lskkz/txtorcon/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-v20lskkz/txtorcon/setup.py"", line 4\n print ""WARNING: not using PyPi over SSH!""\n ^\n SyntaxError: Missing parentheses in call to 'print'. Did you mean print(""WARNING: not using PyPi over SSH!"")?\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n",['ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.'],CheckTheLogsError,NO
178,pycurl,7.19.0.1,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-phjzg8ik/pycurl/setup.py'""'""'; __file__='""'""'/tmp/pip-install-phjzg8ik/pycurl/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-phjzg8ik/pycurl/pip-egg-info\n cwd: /tmp/pip-

In [38]:
split_log(error_df['message'][18])

['Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:',
 '     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-0_60gcb_/astropy/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-0_60gcb_/astropy/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-install-0_60gcb_/astropy/pip-egg-info',
 '         cwd: /tmp/pip-install-0_60gcb_/astropy/',
 '    Complete output (20 lines):',
 '    running egg_info',
 '    creating /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info',
 '    writing /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info/PKG-INFO',
 '    writing dependency_links to /tmp/pip-install-0_60gcb_/astropy/pip-egg-info/astropy.egg-info/dependency_links.txt',
 '    wri

##### Missing gcc  (this is very important here to know)